### Controller
Controller is a class annotated with `@Controller` which handles the request sent to it by Dispatcher Servlet. A typical controller:

In [ ]:
@Controller
public class LoginController{
    
    @PostMapping("/login")
    public String doLogin(@RequestBody LoginData data, Model model){
        // ...
        return "login";
    }
}

Controllers extensively make use of annotations. How the request is processed and how response is prepared is influenced by:
- Annotations (method annotation, parameter annotation, type annotation, etc)
- Parameter type
- Return value

### Request Data

**Get request parameter:** for below method to run, request should be sent to `/param?foo=Bar`

In [ ]:
@GetMapping("param")
@ResponseBody
public String withParam(@RequestParam String foo) {
    return "Obtained 'foo' query parameter value '" + foo + "'";
}

If we want the request parameter and method parameter name to vary:

In [ ]:
@GetMapping("param")
@ResponseBody
public String withParam(@RequestParam("foo") String val) {
    return "Obtained 'foo' query parameter value '" + foo + "'";
}

**Request with group of parameters:** we can get group of parameters as a single object. This may require additional library like Jackson. 

In [ ]:
@GetMapping("group")
@ResponseBody
public String withParamGroup(JavaBean bean) {
    return "Obtained parameter group " + bean;
}

In the above example, if the request looks like: `/group?param1=2K&param2=Ubisoft&param3=ID`, then the JavaBeans object must be:

In [ ]:
public class JavaBean {

    private String param1;
    private String param2;
    private String param3;
    
    // Getter and setters
}

**Path variables:** for below method to run, request should be sent to `/path/hello`

In [ ]:
@GetMapping("path/{var}")
@ResponseBody
public String withPathVariable(@PathVariable String var) {
    return "Obtained 'var' path variable value '" + var + "'";
}

Type conversion also automatically happens

In [ ]:
@GetMapping(path = "/home/add/{a}/{b}")
@ResponseBody
public String add(@PathVariable int a, @PathVariable int b) {
    return new String((a + b) + "");
}

**Get request header:**

In [ ]:
@GetMapping("header")
@ResponseBody
public String withHeader(@RequestHeader String Accept) {
    return "Obtained 'Accept' header '" + Accept + "'";
}

**Get cookies:**

In [ ]:
@GetMapping("cookie")
@ResponseBody
public String withCookie(@CookieValue String openid_provider) {
    return "Obtained 'openid_provider' cookie '" + openid_provider + "'";
}

**Get request body:**

In [ ]:
@PostMapping("body")
@ResponseBody
public String withBody(@RequestBody String body) {
    return "Posted request body '" + body + "'";
}

**Get entity:** entity is body + headers

In [ ]:
@PostMapping("entity")
@ResponseBody
public String withEntity(HttpEntity<String> entity) {
    return "Posted request body '" + entity.getBody() + "'; headers = " + entity.getHeaders();
}

**Standard Servlet Objects**

In [ ]:
@GetMapping("/data/standard/request")
@ResponseBody
public String standardRequestArgs(HttpServletRequest request, Principal user, Locale locale) {
    StringBuilder buffer = new StringBuilder();
    buffer.append("request = ").append(request).append(", ");
    buffer.append("userPrincipal = ").append(user).append(", ");
    buffer.append("requestLocale = ").append(locale);
    return buffer.toString();
}

@PostMapping("/data/standard/request/reader")
@ResponseBody
public String requestReader(Reader requestBodyReader) throws IOException {
    return "Read char request body = " + FileCopyUtils.copyToString(requestBodyReader);
}

@PostMapping("/data/standard/request/is")
@ResponseBody
public String requestReader(InputStream requestBodyIs) throws IOException {
    return "Read binary request body = " + new String(FileCopyUtils.copyToByteArray(requestBodyIs));
}

@GetMapping("/data/standard/session")
@ResponseBody
public String session(HttpSession session) {
    StringBuilder buffer = new StringBuilder();
    buffer.append("session=").append(session);
    return buffer.toString();
}

### Request Matching
**General request mapping:**

In [ ]:
@RequestMapping(path="/simple")
public String simple() {
    return "Maps all HTTP methods";
}

Spring MVC provides annotations specific to HTTP methods: `@GetMapping`, `@PostMapping`, `@PutMapping`, `@DeleteMapping`, etc.

**Presence or absence of params:**

In [ ]:
@GetMapping(path="/mapping/parameter", params="foo")
public String byParameter() {
    return "Mapped by path + method + presence of query parameter!";
}

@GetMapping(path="/mapping/parameter", params="!foo")
public String byParameterNegation() {
    return "Mapped by path + method + not presence of query parameter!";
}

**Presence or absence of header:**

In [ ]:
@GetMapping(path="/mapping/header", headers="FooHeader=foo")
public String byHeader() {
    return "Mapped by path + method + presence of header!";
}

@GetMapping(path="/mapping/header", headers="!FooHeader")
public String byHeaderNegation() {
    return "Mapped by path + method + absence of header!";
}

@RequestMapping can also be applied at class level

In [ ]:
@Controller
@RequestMapping("/login")
public class LoginController{
    // ...
}

### Custom Argument Resolver
Whenever we pass in an argument to the controller method, it uses a `HandlerMethodArgumentResolver` to extract data from it. For example, when we use an argument annotated with `@CookieValue`, the argument resolver parses the request and returns cookie. Similar process occurs if we pass an object. We can create our own custom argument resolver. For example `@RequestAttribute` is not part of Spring MVC. So we create a custom argument resolver:

In [ ]:
// RequestAttribute.class
@Target(ElementType.PARAMETER)
@Retention(RetentionPolicy.RUNTIME)
@Documented
public @interface RequestAttribute {
    String value();
}

In [ ]:
// CustomArgumentResolver.java
// We need to register this in our bean configuration file
public class CustomArgumentResolver implements HandlerMethodArgumentResolver {

    public boolean supportsParameter(MethodParameter parameter) {
        return parameter.getParameterAnnotation(RequestAttribute.class) != null;
    }

    public Object resolveArgument(MethodParameter parameter, ModelAndViewContainer mavContainer,
            NativeWebRequest webRequest, WebDataBinderFactory binderFactory)
            throws Exception {

        RequestAttribute attr = parameter.getParameterAnnotation(RequestAttribute.class);
        return webRequest.getAttribute(attr.value(), WebRequest.SCOPE_REQUEST);
    }

}

In [ ]:
// AppConfig.java
@Configuration
@ComponentScan(basePackages = "org.springframework.samples.mvc")
@EnableWebMvc
public class AppConfig implements WebMvcConfigurer{
    @Override
    public void addArgumentResolvers(List<HandlerMethodArgumentResolver> resolvers) {
        resolvers.add(new CustomArgumentResolver());
    }
}

Notice the `@ComponentScan` line : this enables Spring to find all controllers. `WebMvcConfigurer` makes it easy to register our custom implementations for a number of different things.

In [ ]:
@GetMapping("/data/custom")
@ResponseBody
public String custom(@RequestAttribute("foo") String foo) {
    return "Got 'foo' request attribute value '" + foo + "'";
}

### Response
If we mark the return type (or the method) with `@ResponseBody`, Spring MVC serializes the returned object and sends it directly to the client. The response is a JSON object if appropriate library is in path.

In [ ]:
@PostMapping(path = "/home/colors", produces = MediaType.APPLICATION_JSON_VALUE)
public @ResponseBody ArrayList<Color> getColor() {
    Color red = new Color("red", "#f00");
    Color green = new Color("green", "#0f0");
    Color blue = new Color("blue", "#00f");

    ArrayList<Color> colors = new ArrayList<Color>();
    colors.add(red);
    colors.add(green);
    colors.add(blue);

    return colors;
}

Behind the scenes, a `HttpMessageConverter` underpins reading the request body and generating the response. Multiple converters may be registered for different content types. For `@RequestBody`, the first converter that can read the POSTed "Content-Type" into the desired method parameter type is used. For `@ResponseBody`, the first converter that can write the method return type into one of the client’s "Accept"ed content types is used.  

Different message converters (available after we add @EnableWebMvc):
- `StringHttpMessageConverter`: reads "text/*" into Strings; writes Strings as "text/plain"
- `FormHttpMessageConverter`: reads "application/x-www-form-urlencoded" into `MultiValueMap<String, String>` ; writes `MultiValueMap<String, String>` into "application/x-www-form-urlencoded"
- `ByteArrayMessageConverter`: reads "*/*" into a `byte[]`; writes Objects as "application/octet-stream"
- `Jaxb2RootElementHttpMessageConverter`: reads "text/xml" or "application/xml" into Objects annotated by JAXB annotations; writes JAXB-annotated Objects as "text/xml" or "application/xml". Only registered by default if JAXB is present on the classpath.
- `MappingJacksonHttpMessageConverter`: reads "application/json" into Objects; writes Objects as "application/json"; delegates to the Jackson JSON Processing Library. Only registered by default if Jackson API is in your classpath.

If we just return a string from controller method, that indicates we want to render a view instead. 